In [1]:
import requests
import pandas as pd
import json

In [2]:
APIKEY = open('GoogleAPIKey.txt').read()

In [6]:
def get_image(Address,satellite=False):
    url = 'https://maps.googleapis.com/maps/api/staticmap'
    params = {
        'key':APIKEY,
        'center':Address,
        'zoom':16,
        'size':'800x800',
        'maptype':'satellite' if satellite else 'roadmap',
    }
    headers = {
    }
    r = requests.get(url, params=params,headers=headers)
    r.raise_for_status()
    temp = r.content
    folder = 'satellite_view' if satellite else 'roadmap_view'
    with open(f'../{folder}/{Address.strip()}.png','wb') as file:
        file.write(temp)

In [4]:
df_universitys = pd.read_csv('../data/world-university-rankings/cwurData.csv')
df_universitys = df_universitys[df_universitys['year']==2015]
universities = df_universitys.institution.to_list()


In [8]:
for i,university in enumerate(universities):
    # skipped University of Eastern Piedmont Amedeo Avogadro""
    if i == 629:
        university = university.rstrip('"')
    try:
        get_image(university,satellite=True)
    except Exception as e:
        print(e)
        print(i)
    break

In [9]:
for i,university in enumerate(universities):
    # skipped University of Eastern Piedmont Amedeo Avogadro""
    if i == 629:
        university = university.rstrip('"')
    try:
        get_image(university)
    except Exception as e:
        print(e)
        print(i)
    break

In [10]:
import cv2
import numpy as np

def count_water(img):
    count=0
    for row in img:
        for pixel in row:
            if (pixel == [255,218,170]).all():
                count+=1

    return count >= 25000

def count_park(img):
    count=0
    for row in img:
        for pixel in row:
            if (pixel == [197,232,197]).all():
                count+=1

    return count >= 40000

def get_green_pct(re_img1):
    b, g, r = cv2.split(re_img1)

    ttl = re_img1.size / 3 #divide by 3 to get the number of image PIXELS

    """b, g, and r are actually numpy.ndarray types,
    so you need to use the appropriate method to sum
    all array elements"""
    B = float(np.sum(b)) / ttl #convert to float, as B, G, and R will otherwise be int
    G = float(np.sum(g)) / ttl
    R = float(np.sum(r)) / ttl
    return '{:.2%}'.format(G/(R+G+B))

In [11]:
df_universitys.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1200,1,Harvard University,USA,1,1,1,1,1,1,1,1.0,3,100.00,2015
1201,2,Stanford University,USA,2,9,2,4,5,3,3,4.0,10,98.66,2015
1202,3,Massachusetts Institute of Technology,USA,3,3,11,2,15,2,2,2.0,1,97.54,2015
1203,4,University of Cambridge,United Kingdom,1,2,10,5,11,6,12,13.0,48,96.81,2015
1204,5,University of Oxford,United Kingdom,2,7,13,10,7,12,7,9.0,15,96.46,2015


In [12]:
output = {}

for i,row in df_universitys.iterrows():
    university = row['institution']
    temp = {
        'world_ranking':row['world_rank']
    }

    try:
        img = cv2.imread(f"satellite_view/{university}.png")
        temp['GreenPct'] = get_green_pct(img)
    except:
        temp['GreenPct'] = 'NA'

    try:
        img = cv2.imread(f"roadmap_view/{university}.png")
        temp['IsNearWater'] = count_water(img)
    except:
        temp['IsNearWater'] = 'NA'


    try:
        img = cv2.imread(f"roadmap_view/{university}.png")
        temp['IsNearParks'] = count_park(img)
    except:
        temp['IsNearWater'] = 'NA'
        
    output[university] = temp
    break

In [24]:
with open('../data/GoogleMapsAndUniversityFeatures.json','w') as file:
    file.write(json.dumps(output))

In [9]:

img = cv2.imread(f"roadmap_view/nanjing university of chinese medicine.png")

In [10]:

count=0
for row in img:
    for pixel in row:
        if (pixel == [197,232,197]).all():
            count+=1

In [11]:
print(count)

42296
